# About: NFSの準備

CoursewareHub用のNFSを準備する

* NFSサーバーの設定
  * NFS serverのインストール
  * CoursewareHubのノードへのexportを設定
* CousewareHubのノードにNFSマウントを設定
  * NFS clientのインストール
  * fstabの設定

# Notebookと環境のBinding

Inventoryファイルと、Inventory中のgroup名でBind対象ホストを指示する。


- `hosts_file`: Inventoryファイル
- `target_group`: Ansible Inventory中に定義されているグループ( `[グループ名]` のように`[]`で囲まれた定義 )のうち、CoursewareHub構築に使うグループ名を指定します。


In [ ]:
hosts_file = './cwhtest0001_inventory'
target_group = 'cwhtest0001'

# 接続確認
%env ANSIBLE_INVENTORY={hosts_file}
!ansible -m ping {target_group}

NFSサーバーを含むAnsibleのgroup名を指定する。対象が正しいか、Ansible pingモジュールで動作確認する。

In [ ]:
target_server = 'cwhub_nfs_server'

!ansible -m ping {target_server} -l {target_group}

clientとするnodeはAnsibleのgroup名で定義されている。対象が正しいか、Ansible pingモジュールで動作確認する。

In [ ]:
import json
import re
result = !ansible -m debug -a 'msg={{{{ nfsnodes_group }}}}' {target_server}
firstline_pattern = re.compile(r'^.*\s*\=\>\s*({.*)$')
result = '\n'.join([firstline_pattern.match(result[0]).group(1)] + result[1:])
target_nodes = json.loads(result)['msg']

!ansible -m ping {target_nodes}

各々のノードには、servicenet_ip変数が定義されている必要がある。

In [ ]:
!ansible -m debug -a 'msg={{{{ servicenet_ip }}}}' {target_server} -l {target_group}
!ansible -m debug -a 'msg={{{{ servicenet_ip }}}}' {target_nodes} -l {target_group}

デフォルトユーザを取得しておく。

In [ ]:
result = !ansible -a whoami {target_server}
default_user = result[1]
default_user

# Playbookの準備

In [ ]:
import tempfile
work_dir = tempfile.mkdtemp()
work_dir

In [ ]:
!rm -fr {work_dir}/jupyterhub-deploy
!git clone https://github.com/NII-cloud-operation/CoursewareHub-LC_jupyterhub-deploy.git {work_dir}/jupyterhub-deploy

Check the roles...

In [ ]:
!ls -la {work_dir}/jupyterhub-deploy/roles

In [ ]:
!cd {work_dir}/jupyterhub-deploy/ && git log -n 5

# NFS serverのインストールと設定

NFSサーバーとCoursewareHubのmasterノードが共用の場合は、以下のAnsible roleを使用する。

`nfs_server_roles = ['nfs_server']`

NFSサーバーとCousewareHubのmasterノードを分離する場合は、以下のAnsible roleを使用する。

`nfs_server_roles = ['nfs_server_dedicated']`

In [ ]:
msg_stdout = !ansible -m debug -a 'msg={{{{ nfs_server_role }}}}' {target_server} -l {target_group}
nfs_server_role = json.loads(''.join([msg_stdout[0].split()[-1]] + msg_stdout[1:]))['msg']
nfs_server_role

# nfs_server_roles = ['nfs_server']
# nfs_server_roles = ['nfs_server_dedicated']

nfs_server_roles = [ nfs_server_role ]
nfs_server_roles

In [ ]:
import os
import yaml
with open(os.path.join(work_dir, 'jupyterhub-deploy/deploy-nfs-server.yml'), 'w') as f:
    f.write(yaml.dump([{'hosts': target_server,
                       'become': True,
                       'roles': nfs_server_roles}]))
!cat {work_dir}/jupyterhub-deploy/deploy-nfs-server.yml

Now, dry-run first...

In [ ]:
!ansible-playbook -CDv {work_dir}/jupyterhub-deploy/deploy-nfs-server.yml -l {target_group} ;:

It seems OK, apply it to hosts.

In [ ]:
!ansible-playbook {work_dir}/jupyterhub-deploy/deploy-nfs-server.yml -l {target_group}

# firewalldの設定

In [ ]:
!ansible -CDv -b -m ansible.posix.firewalld -a 'zone=public service=nfs permanent=yes state=enabled' {target_server} -l {target_group}

In [ ]:
!ansible -b -m ansible.posix.firewalld -a 'zone=public service=nfs permanent=yes state=enabled' {target_server} -l {target_group}

In [ ]:
!ansible -CDv -b -m ansible.posix.firewalld \
    -a 'zone=public service=nfs immediate=yes permanent=yes state=enabled' {target_server} -l {target_group}

In [ ]:
!ansible -b -m ansible.posix.firewalld \
    -a 'zone=public service=nfs immediate=yes permanent=yes state=enabled' {target_server} -l {target_group}

# NFS clientのインストールと設定

In [ ]:
with open(os.path.join(work_dir, 'jupyterhub-deploy/deploy-nfs-client.yml'), 'w') as f:
    f.write(yaml.dump([{'hosts': target_nodes,
                       'become': True,
                       'roles': ['nfs_client']}]))
!cat {work_dir}/jupyterhub-deploy/deploy-nfs-client.yml

In [ ]:
!ansible-playbook -CDv {work_dir}/jupyterhub-deploy/deploy-nfs-client.yml -l {target_group}

OK, apply it.

In [ ]:
!ansible-playbook {work_dir}/jupyterhub-deploy/deploy-nfs-client.yml -l {target_group}

# NFSの動作確認


In [ ]:
!ansible -a 'df -h' {target_nodes} -l {target_group}

ファイルが見えるか、確認する。

In [ ]:
!ansible -a 'ls -la {{{{ nfspath }}}}/exchange' {target_server} -l {target_group}
!ansible -a 'ls -la /exchange' {target_nodes} -l {target_group}

## 動作確認その1

Create a test file on `/exchange`...

In [ ]:
!ansible -b -a 'touch {{{{ nfspath }}}}/exchange/test' {target_server} -l {target_group}

Check whether `exchange` directory of hub and nodes are sync.

In [ ]:
!ansible -a 'ls -la {{{{ nfspath }}}}/exchange' {target_server} -l {target_group}
!ansible -a 'ls -la /exchange' {target_nodes} -l {target_group}

OK, the test file also appears in NFS clients.

Remove the test file.

In [ ]:
!ansible -b -a 'rm {{{{ nfspath }}}}/exchange/test' {target_server} -l {target_group}

In [ ]:
!ansible -a 'ls -la {{{{ nfspath }}}}/exchange' {target_server} -l {target_group}
!ansible -a 'ls -la /exchange' {target_nodes} -l {target_group}

## 再起動しても動く？

Reboot VM... NFS clientだけ。

NFS serverは再起動しない

In [ ]:
!ansible -b -m reboot {target_nodes} -l {target_group}

## 動作確認その2

再起動後でも動くか？

In [ ]:
!ansible -b -a 'touch {{{{ nfspath }}}}/exchange/test' {target_server} -l {target_group}

In [ ]:
!ansible -a 'ls -la {{{{ nfspath }}}}/exchange' {target_server} -l {target_group}
!ansible -a 'ls -la /exchange' {target_nodes} -l {target_group}

In [ ]:
!ansible -b -a 'rm {{{{ nfspath }}}}/exchange/test' {target_server} -l {target_group}

In [ ]:
!ansible -a 'ls -la {{{{ nfspath }}}}/exchange' {target_server} -l {target_group}
!ansible -a 'ls -la /exchange' {target_nodes} -l {target_group}

# 後始末

In [ ]:
!rm -rf {work_dir}